**<html><p style="font-size:30px;">Project Assignment B:</p></html>**<br>
**<html><p style="font-size:45px;color:orange;">NYC Restaurant Inspections</p></html>**<br>

# 1. Motivation

#### *What is your dataset?*

Our dataset consists of results of restaurant inspections in New York and is produced by the Department of Health and Mental Hygiene (DOHMH). It contains every sustained or not yet adjudicated violation citation from every full or special program inspection conducted up to three years prior to the most recent inspection for restaurants and college cafeterias in an active status on the date of the data pull. For the purpose of this project the whole dataset was utilized, ranging from 2011 up to 2023. 

#### *Why did you choose this/these particular dataset(s)?*

This particular dataset was selected for a set of reasons. First of all, the subject of the dataset, restaurant inspections, is relevant to everyday life and might be interesting to a great deal of people. New York, as the world’s most famous city and home to probably the greatest variety of ethnic cuisines in the world, has a reputation of an exciting restaurant scene that is a destination by itself. We believe that a showcasing of the evolution of restaurant quality through time and its distribution through different parts of the city would appeal not only to New Yorker’s, but also to potential visitors and even Instagram foodies from all over the world. 

Secondly, the dataset offers a lot of information about restaurant inspections, including type of cuisine, type and severity of violation, region, date etc., giving a clear picture of the actual situation. This plethora of information allows us to combine different elements of the data and transform them into valuable insights for the reader.

#### *What was your goal for the end user's experience?*

In the end, the aim of this article is to capture the reader’s interest and inform about the quality of restaurants in NY in a creative and inventive way, utilizing visualization tools and convening information in an interactive and original manner.

# 2. Basic stats

#### *Write about your choices in data cleaning and preprocessing.*

Because this dataset is compiled from several large administrative data systems, it contains some illogical values that could be a result of data entry or transfer errors. Data may also be missing. For this reason certain elements were removed or rearranged to prepare the dataset before creating the plots. First of all, certain restaurants that are in the dataset have not been inspected yet and show no results. These restaurants are noted as having an inspection date of 1/1/1900 and were observed as outliers in the bar plot below before being removed. Even though the dataset ranges between 2011 - 2023, it can be seen in the bar plots below that there is a very small number of occurrences for the years prior to 2018. This is because old inspections stay on the list only as long as there is a new inspection in the same restaurant in the last three years. For this reason, for the purpose of this project we focused only on inspections that happened after 2018.

Another source of outliers was the notation "0" in the borough column, meaning that some restaurants were not registered with a correct location and as a result they were removed. Some columns that did not convey any valuable information for this project, such as 'Community Board','Council District','Census Tract' etc. were also removed. The null values in the dataset were investigated in orded to find out their amount in each column and if they interfere with the results. The different types of restaurants/cuisines were specified and those categories that comprised of a very small number of entries were dropped, as they were to small to help reach meaningful conclusions.

#### *Write a short section that discusses the dataset stats, containing key points/plots from your exploratory data analysis.*

After the data were cleaned, the number of total occurrences per year and their seasonality were investigated in order to get an idea of progress through time. The number of investigations per borough was also calculated to look for spatial patterns regarding the distribution of investigations and violations. This information was combined with the number of restaurants of each different type of cuisine to get interesting insights about the concentration of violations in each category, the distribution of grades and the types of violations.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df_inspec = pd.read_csv('DOHMH_New_York_City_Restaurant_Inspection_Results.csv')

# Show dimensions
print(f'Our dataset has {df_inspec.shape[0]} rows and {df_inspec.shape[1]} columns\n')

# Show column info
df_inspec.info()

In [ ]:
# Summary statistics
df_inspec.describe()

## 2.1. Removing outliers

In [ ]:
from matplotlib.pyplot import figure

# Create column 'Year' from 'INSPECTION DATE'
df_inspec['Year'] = [date.split('/')[-1] for date in df_inspec.loc[:,'INSPECTION DATE']]

# Convert the 'Year' column to an integer type
df_inspec['Year'] = df_inspec['Year'].astype(int)

data = df_inspec.groupby('Year')['CAMIS'].count()

figure(figsize=(4, 3), dpi=80)
plt.bar(data.index, data.values, color='red', width=0.7)
plt.title('Occurrences per year')
plt.show()

In [ ]:
# Exclude the year 1900, we only want rows with accurate dates
df_inspec_filtered = df_inspec[df_inspec['Year'] != 1900].copy()

#Find the minimum and maximum year
min_year = df_inspec_filtered['Year'].min()
max_year = df_inspec_filtered['Year'].max()

#Calculate the range of years
year_range = max_year - min_year

print(f"Minimum year is {min_year}")
print(f"Maximum year is {max_year}")
print(f"The range of the years is {year_range}")

data = df_inspec_filtered.groupby('Year')['CAMIS'].count()

figure(figsize=(4, 3), dpi=80)
plt.bar(data.index, data.values, color='green', width=0.7)
plt.title('Occurrences per year')
plt.show()

In [ ]:
df_inspec_filtered = df_inspec[df_inspec['Year'] > 2018].copy()

In [ ]:
data = df_inspec_filtered.groupby('BORO')['CAMIS'].count()

figure(figsize=(7, 3), dpi=80)
plt.bar(data.index, data.values, color='red', width=0.7)
plt.title('Occurrences per borough')
plt.show()

In [ ]:
# Exclude the BORO '0'
df_inspec_filtered = df_inspec_filtered[df_inspec_filtered['BORO'] != '0'].copy()

data = df_inspec_filtered.groupby('BORO')['CAMIS'].count()

figure(figsize=(7, 3), dpi=80)
plt.bar(data.index, data.values, color='green', width=0.7)
plt.title('Occurrences per borough')
plt.show()

## 2.2. Choosing relevant columns

In [ ]:
# List of columns we will not be working with
useless_cols = ['Community Board','Council District','Census Tract','BIN','BBL','NTA','Location Point1']

df_inspec_filtered.drop(useless_cols, axis=1, inplace=True)

## 2.2.1. Adding interesting columns

In [ ]:
import geopandas as gpd
from shapely.geometry import Point

neighborhoods = gpd.read_file('Neighborhood Boundaries.geojson')

# Create a Point geometry column from the 'Latitude' and 'Longitude' columns
geometry = [Point(xy) for xy in zip(df_inspec_filtered['Longitude'], df_inspec_filtered['Latitude'])]

# Create a GeoDataFrame from the restaurants DataFrame and the Point geometry
restaurants_gdf = gpd.GeoDataFrame(df_inspec_filtered, geometry=geometry, crs=neighborhoods.crs)

# Perform the spatial join
joined = gpd.sjoin(restaurants_gdf, neighborhoods, predicate='within')

# Create a new column 'NEIGHBORHOOD' in the restaurants DataFrame with the neighborhood names
df_inspec_filtered['NEIGHBORHOOD'] = joined['ntaname']

In [ ]:
# Parse inspection date
df_inspec_filtered['INSPECTION DATE'] = pd.to_datetime(df_inspec['INSPECTION DATE'])

# Extract yearmonth from parsed inspection date
df_inspec_filtered['YEARMONTH'] = df_inspec_filtered['INSPECTION DATE'].dt.strftime('%Y-%m')

In [ ]:
# Since we have a lot of null GRADES
# we will add a column SCORE_GRADE that is based on the logic provided by the owner of the dataset
import numpy as np

def assign_grade(score):
    if score < 14:
        return 'A'
    elif score >= 14 and score <= 27:
        return 'B'
    elif score >= 28:
        return 'C'
    else:
        return None

df_inspec_filtered['SCORE_GRADE'] = df_inspec_filtered['SCORE'].apply(assign_grade)

In [ ]:
# Distribution of restaurant types
group = df_inspec_filtered.groupby('CUISINE DESCRIPTION')['CAMIS'].count().sort_values(ascending=False)
display(group)

In [ ]:
# Distribution of grades
group = df_inspec_filtered.groupby('GRADE')['CAMIS'].count().sort_values(ascending=False)
display(group)
criticalgraph = df_inspec_filtered[["CRITICAL FLAG", "GRADE"]]
critnona = criticalgraph.dropna()
critnona["GRADE"].value_counts().plot(kind="pie")

In this pie-chart it is evident that the vast majority of inspections end up in an A-grade evaluation. Even though pie charts can sometimes be a little tricky to interpret, big differences like the one illustrated here can easily be higlighted with this type of visualization.

In [ ]:
# Distribution of calculated grades
group = df_inspec_filtered.groupby('SCORE_GRADE')['CAMIS'].count().sort_values(ascending=False)
display(group)

In [ ]:
import seaborn as sns

# KDE plot of the score
fig, ax = plt.subplots()

sns.kdeplot(df_inspec_filtered["SCORE"], fill=True, ax=ax)

fig.suptitle("Inspection score density")

plt.show()

The probability density plot for each possible score successfully illustrates that most inspections end up with a score between 0 and 15 and that only a small percentage ends up having very big score that can lead to a closure.

In [ ]:
# Unique actions
actions = df_inspec_filtered['ACTION'].unique()
actions

In [ ]:
# Top 20 Violation Descriptions
group = df_inspec_filtered.groupby('VIOLATION DESCRIPTION')['CAMIS'].count().sort_values(ascending=False)[:20]
display(group)

## 2.3. Season plots

In [ ]:
# Create a new column 'season' based on the inspection date
df_inspec_filtered['season'] = pd.cut(
    x=df_inspec_filtered['INSPECTION DATE'].dt.month,
    bins=[0, 3, 6, 9, 12],
    labels=['winter', 'spring', 'summer', 'fall'],
    include_lowest=True
)
# Group the closed restaurants by season and count the number of closed restaurants
season_counts = df_inspec_filtered.groupby('season').size()

# Create a bar plot of the number of closed restaurants by season
season_counts.plot(kind='bar', rot=0, color='dodgerblue')
plt.xlabel('Season')
plt.ylabel('Number of Inspections')
plt.title('Number of Inspections by Season')
plt.show()

This barplot shows that in the years in question, the season in which a restaurant has the highest chance of being inspected is the winter, with the rest of the seasons being relatively the same. It should be taken into account that the interruption caused to everyday life by the pandemic and the quarantine may give a false impression and create a pattern that does not correspond to a normal year.

In [ ]:
# Create new dataframe with only closed restaurants
closed_restaurants_df = df_inspec_filtered.loc[df_inspec_filtered['ACTION'] == 'Establishment Closed by DOHMH. Violations were cited in the following area(s) and those requiring immediate action were addressed.'].sort_values('Year', ascending=True)

# Create a new column 'season' based on the inspection date
closed_restaurants_df['season'] = pd.cut(
    x=closed_restaurants_df['INSPECTION DATE'].dt.month,
    bins=[0, 3, 6, 9, 12],
    labels=['winter', 'spring', 'summer', 'fall'],
    include_lowest=True
)
# Group the closed restaurants by season and count the number of closed restaurants
season_counts = closed_restaurants_df.groupby('season').size()

# Create a bar plot of the number of closed restaurants by season
season_counts.plot(kind='bar', rot=0, color='tomato')
plt.xlabel('Season')
plt.ylabel('Number of Closed Restaurants')
plt.title('Number of Closed Restaurants by Season')
plt.show()


In [ ]:
# Season plot for each year

# Create a new column 'season' based on the inspection date
closed_restaurants_df['season'] = pd.cut(
    x=closed_restaurants_df['INSPECTION DATE'].dt.month,
    bins=[0, 3, 6, 9, 12],
    labels=['winter', 'spring', 'summer', 'fall'],
    include_lowest=True
)

# Group the closed restaurants by year and season and count the number of closed restaurants
year_season_counts = closed_restaurants_df.groupby(['Year', 'season']).size()

# Get a list of unique years in the data
years = year_season_counts.index.get_level_values('Year').unique()

# Reindex the year_season_counts DataFrame to include all seasons for all years
all_seasons = pd.MultiIndex.from_product([years, ['winter', 'spring', 'summer', 'fall']], names=['Year', 'season'])
year_season_counts = year_season_counts.reindex(all_seasons, fill_value=0)

# Create a separate bar plot for each year, with two plots per row
num_rows = len(years) // 2 + len(years) % 2
fig, axs = plt.subplots(num_rows, 2, figsize=(8, 2*num_rows), squeeze=False)
for i, year in enumerate(years):
    row = i // 2
    col = i % 2
    year_counts = year_season_counts.loc[year]
    year_counts.plot(kind='bar', rot=0, color='tomato', ax=axs[row, col])
    axs[row, col].set_xlabel('Season')
    axs[row, col].set_ylabel('Number of Closings')
    axs[row, col].set_title(f'Number of Closed Restaurants on {year}')

plt.tight_layout()
plt.show()


## 2.4. Boxplots of score by cuisine type

In [ ]:
# Let us look at the scores by cuisine
score_cuisine = pd.concat([df_inspec_filtered['CUISINE DESCRIPTION'], df_inspec_filtered['SCORE']], axis=1)

# Count occurrences of each cuisine type
cuisine_counts = score_cuisine['CUISINE DESCRIPTION'].value_counts()

# Sort cuisine types by count in descending order
cuisine_order = cuisine_counts.sort_values(ascending=False).index.tolist()

# Create boxplot with sorted cuisine types
f, ax = plt.subplots(figsize=(16, 8))
fig = sns.boxplot(x='CUISINE DESCRIPTION', y='SCORE', data=score_cuisine, order=cuisine_order)
plt.xticks(rotation=90)

plt.show()

In this very nice boxplot, the average score of each type of cuisine is illustrated. The types of cuisine are arranged alphabetically and the distribution of their score, along with the average ar shown. Some outliers with very high scores indicate that in some restaurants more than a handful of serious violations were recorded.

## 2.5. Count of grades per borough

In [ ]:
sns.countplot(y='GRADE',data=df_inspec_filtered,hue='BORO',palette='Greens_d',order=['A','B','C','N','P','Z'])
plt.style.use("fivethirtyeight")

In this barplot we can see that distribution of grades in each borough is relatively similar, with the absolute number of each grading in each borough corresponding with the total number of inspections in each borough.

# 3. Data Analysis 

#### *Describe your data analysis and explain what you've learned about the dataset.*

The first thing we looked for is the distribution of inspections among districts. New York is divided in "boroughs", so with the help of a relevant geojson file we counted the number of inspections, types of violations and rates of violations and closures in each borough. The results of this analysis indicate that the absolute number of inspections and violations vary greatly between boroughs, with more than ten times more inspections taking place in Manhattan compared to Staten Island. The rate at which inspections end up having a negative outcome was found to be very similar between boroughs, and very high in general, being generally greater than 98%. It should be noted that this includes all types of possible violations, some of them being a bit trivial, e.g. not having a posted "Wash Your Hands" sign in the restroom. The number of inspections ending up in the closure of a restaurant was also calculated for each borough, and the rate of closures per inspection was found to be greater in Brooklyn than other areas.

A closer look was taken on the grading system, the score achieved by each restaurant and the types of violations. In general, most inspections end up in an A grade, which is the best possible grade and means that no reinspection is needed for the restaurant. The probability density of each score was calculated, again strengthening the conclusion that even though most inspections end up finding violations, the vast majority ascore less than 13 points, meaning that the violations were not very important and that they receive an A grade.

This analysis about the spatial distribution of inspections was taken one step further by repeating the previous steps for smaller areas (neighborhoods), in order to get a more detailed opinion about locations of violations. The "Midtown-Midtown South" neighborhood of Manhattan sticks out at first, with significantly more violations occurring there than any other neighborhood in NY. Upon a closer inspection however, it is observed that the rate of inspections ending up finding violations is not greater there than the rest of the city, leading to the conclusion that this neighborhood is probably dense with restaurants and therefore sees more inspections in general. Regarding inspections that end up in closures, it is notable that in the "Grymes Hill-Clifton-Fox Hills" neighborhood in Staten Island almost 20% of inspections ended up in the closure of the restaurant! 

After that, we focused on how inspections progressed through time for the period of interest. Here the most important, and admittedly expected, thing, is the total absence of inspections during the peak of the corona-virus pandemic. It is a fact that New York was hit very hard by COVID-19, being the city facing the most difficulties in the USA. Between the middle of March 2020 and July 2021, for the greatest part of which time restaurants were closed, almost no inspections took place, leaving a huge gap in the calendar analysis. In an attempt to compensate(?) for the lost time, inspections skyrocketed the following year, with more than three-times more inspections taking place in 2022 than 2019! This increase in the number of inspections holds true for all types of cuisines, with hunderds of inspections occuring almost every day in 2022 and even more moving into 2023. Looking in the number of inspections per season, we found that there are significantly more inspections taking place in the winter compared to other seasons, with the number of closures being proportional to the total inspections. The types of violations were analysed to see their progression through time. The result was that generally, the number of each type of violation spotted is proportional to the total number of inspections.

Moving forward, the distribution of inspections, violations and grades between the different types of cuisines was investigated. The distribution of grades was found to be relatively similar for all types of cuisines, with the majority receiving an A in most types. Chinese, Indian, Carribean and Jewish/Kosher restaurants have the lowest rate of A-grades among cuisines, with Donuts and Hamburgers standing out with the highest percentage of A-grades. 


## 4. Genre

The Genre that was chosen for this project is the Magazine Style genre. Like in the previous assignment, This form of narrative visualization was chosen because it allows great freedom in organizing the visuals and accompanying text in a way that is easy to read and most importantly is familiar to the reader. In terms of visual narrative, the article is characterised by a Consistent Visual Platform and Zooming can be used to highlight some details, e.g. in the choropleth maps. Some Motion is also introduced with gifs to make the narration lighter and easier to follow. The Narrative Structure is Linear, with a clear order of the elements that are analyzed, while Interactivity is strongly present with interactive bar plots and scatter maps. Messaging is also achieved by having an introductory text and some accompanying captions to gifs that help communication with the reader.

## 5. Visualizations

### 5.1.1. Boroughs Choropleth

In [ ]:
# All the maps shown should have the same center
NYC_CENTER = {"lat": 40.718678, "lon": -73.988798}

In [ ]:
# Dataframe with negative actions
df_negative_outcome = df_inspec_filtered.loc[~df_inspec_filtered['ACTION'].isin([
    'No violations were recorded at the time of this inspection.',
    'Establishment re-opened by DOHMH.'])].copy()

# Dataframe with restaurant closures
df_closures = df_inspec_filtered.loc[df_inspec_filtered['ACTION'].isin([
    'Establishment Closed by DOHMH. Violations were cited in the following area(s) and those requiring immediate action were addressed.',
    'Establishment re-closed by DOHMH.'])].copy()

In [ ]:
import plotly.express as px
import json
with open('Borough Boundaries.geojson') as f:
    geodata = json.load(f)
    
map_data = df_negative_outcome.groupby('BORO')['CAMIS'].count().reset_index()

# Make the map
fig = px.choropleth_mapbox(map_data, geojson=geodata, locations='BORO', color='CAMIS',
                           featureidkey="properties.boro_name",
                           color_continuous_scale="RdYlGn_r",
                           range_color=(map_data.CAMIS.min(), map_data.CAMIS.max()),
                           mapbox_style="carto-positron",
                           zoom=9, center = NYC_CENTER,
                           opacity=0.5
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0}, coloraxis_colorbar=dict(title='Negative outcome count'))
fig.show()

In [ ]:
#map_data
df_inspec_filtered.groupby('BORO')['CAMIS'].count().reset_index()

In [ ]:
total_restaurants = [len(df_inspec_filtered[df_inspec_filtered['BORO']==boro]) for boro in map_data['BORO']]

# Adjust data for total of restaurants
map_data['ADJUSTED'] = map_data['CAMIS']/total_restaurants


# Make the map
fig = px.choropleth_mapbox(map_data, geojson=geodata, locations='BORO', color='ADJUSTED',
                           featureidkey="properties.boro_name",
                           color_continuous_scale="RdYlGn_r",
                           range_color=(map_data.ADJUSTED.min(), map_data.ADJUSTED.max()),
                           mapbox_style="carto-positron",
                           zoom=9, center = NYC_CENTER,
                           opacity=0.5
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0}, coloraxis_colorbar=dict(title='Negative outcome rate'))
fig.show()

In [ ]:
import plotly.express as px
import json
with open('Borough Boundaries.geojson') as f:
    geodata = json.load(f)
    
map_data = df_closures.groupby('BORO')['CAMIS'].count().reset_index()

# Make the map
fig = px.choropleth_mapbox(map_data, geojson=geodata, locations='BORO', color='CAMIS',
                           featureidkey="properties.boro_name",
                           color_continuous_scale="RdYlGn_r",
                           range_color=(map_data.CAMIS.min(), map_data.CAMIS.max()),
                           mapbox_style="carto-positron",
                           zoom=9, center = NYC_CENTER,
                           opacity=0.5
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0}, coloraxis_colorbar=dict(title='Closures count'))
fig.show()

In [ ]:
total_restaurants = [len(df_inspec_filtered[df_inspec_filtered['BORO']==boro]) for boro in map_data['BORO']]

# Adjust data for total of restaurants
map_data['ADJUSTED'] = map_data['CAMIS']/total_restaurants

# Make the map
fig = px.choropleth_mapbox(map_data, geojson=geodata, locations='BORO', color='ADJUSTED',
                           featureidkey="properties.boro_name",
                           color_continuous_scale="RdYlGn_r",
                           range_color=(map_data.ADJUSTED.min(), map_data.ADJUSTED.max()),
                           mapbox_style="carto-positron",
                           zoom=9, center = NYC_CENTER,
                           opacity=0.5
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0}, coloraxis_colorbar=dict(title='Closures rate'))
fig.show()

The choropleth maps above illustrate the number of inspections with negative outcome (meaning those that end up finding even one violation), the rate at which these happen, the number of inspections that end up in the closure of the reastaurant, and again the rate at which closure occur. This type of visualization is ideal for showcasing the differences beetween districts, as it get the message through to the reader very quickly through the use of colors.

The same applies to the maps below, only this time the city is divided into smaller part, i.e. neighborhoods. This gives an even clearer picture of reality, since the boroughs are too big and what is true for one part may not be for another. Through the use of neighborhoods, one can easily see the results of inspections in a very specific part of the city and form an opinion about the level of restaurants there.

### 5.1.2. Neighborhoods Choropleth

In [ ]:
import plotly.express as px

map_data = df_negative_outcome.groupby('NEIGHBORHOOD')['CAMIS'].count().reset_index()

# Make the map
fig = px.choropleth_mapbox(map_data, geojson=neighborhoods, locations='NEIGHBORHOOD', color='CAMIS',
                           featureidkey="properties.ntaname",
                           color_continuous_scale="RdYlGn_r",
                           range_color=(map_data.CAMIS.min(), map_data.CAMIS.max()),
                           mapbox_style="carto-positron",
                           zoom=9, center = NYC_CENTER,
                           opacity=0.5
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0}, coloraxis_colorbar=dict(title='Negative outcome count'))
fig.show()

In [ ]:
total_restaurants = [len(df_inspec_filtered[df_inspec_filtered['NEIGHBORHOOD']==n]) for n in map_data['NEIGHBORHOOD']]

# Adjust data for total of restaurants
map_data['ADJUSTED'] = map_data['CAMIS']/total_restaurants


# Make the map
fig = px.choropleth_mapbox(map_data, geojson=neighborhoods, locations='NEIGHBORHOOD', color='ADJUSTED',
                           featureidkey="properties.ntaname",
                           color_continuous_scale="RdYlGn_r",
                           range_color=(map_data.ADJUSTED.min(), map_data.ADJUSTED.max()),
                           mapbox_style="carto-positron",
                           zoom=9, center = NYC_CENTER,
                           opacity=0.5
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0}, coloraxis_colorbar=dict(title='Negative outcome rate'))
fig.show()

In [ ]:
import plotly.express as px

map_data = df_closures.groupby('NEIGHBORHOOD')['CAMIS'].count().reset_index()

# Make the map
fig = px.choropleth_mapbox(map_data, geojson=neighborhoods, locations='NEIGHBORHOOD', color='CAMIS',
                           featureidkey="properties.ntaname",
                           color_continuous_scale="RdYlGn_r",
                           range_color=(map_data.CAMIS.min(), map_data.CAMIS.max()),
                           mapbox_style="carto-positron",
                           zoom=9, center = NYC_CENTER,
                           opacity=0.5
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0}, coloraxis_colorbar=dict(title='Closures count'))
fig.show()

In [ ]:
total_restaurants = [len(df_inspec_filtered[df_inspec_filtered['NEIGHBORHOOD']==n]) for n in map_data['NEIGHBORHOOD']]

# Adjust data for total of restaurants
map_data['ADJUSTED'] = map_data['CAMIS']/total_restaurants

# Make the map
fig = px.choropleth_mapbox(map_data, geojson=neighborhoods, locations='NEIGHBORHOOD', color='ADJUSTED',
                           featureidkey="properties.ntaname",
                           color_continuous_scale="RdYlGn_r",
                           range_color=(map_data.ADJUSTED.min(), map_data.ADJUSTED.max()),
                           mapbox_style="carto-positron",
                           zoom=9, center = NYC_CENTER,
                           opacity=0.5
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0}, coloraxis_colorbar=dict(title='Closures rate'))
fig.show()

### 5.1.3. Cuisine Scattermap

In [ ]:
import plotly.express as px
import math
import plotly.graph_objects as go
import pandas as pd

df_recent = df_inspec_filtered[(df_inspec_filtered['Year'] == 2023)]

focuscuisines = df_recent.groupby('CUISINE DESCRIPTION')['CAMIS'].count().sort_values(ascending=False)[:10].index

for cuisine in focuscuisines:

    df_focus = df_recent[df_recent['CUISINE DESCRIPTION']==cuisine].sort_values('SCORE_GRADE', ascending=True)

    fig = px.scatter_mapbox(df_focus, lat='Latitude', lon='Longitude', color='SCORE_GRADE',
                            color_discrete_map={
                                'A': '#636EFA',
                                'B': '#00CC96',
                                'C': '#FFA15A'
                             },
                            hover_data=['DBA'],
                            mapbox_style="carto-positron", 
                            zoom=9, center = NYC_CENTER)

    fig.update_layout(
        margin=dict(l=10, r=30, t=10, b=10),
        autosize=False,
        width=550,
        height=400,
        legend=dict(
            x=0.01,
            y=0.99,
            traceorder='normal',
            font=dict(
                family='sans-serif',
                size=12,
                color='black'
            ),
            bgcolor='rgba(255,255,255, 0.6)'
        )
    )
    
    display(cuisine)
    fig.show()
    #fig.write_html('output/grademaps/'+cuisine.lower().replace("/", "_")+'.html')

This interactive map can be very useful when being interested in a specific cuisine type. One can easily see the number of restaurants of each category that received each grade and focus on the part of the city that interests him/her, no matter how big or small it is. Note that the map offers the possibility to find exactly the position of each restaurant in the city so that you can directly find the address of a place with a good grade (or bad if you want to see how the restaurant where you ate yesterday performs)!

## 5.2. Calendar plots

In [ ]:
import calplot

# Filter the data and choose the cuisine we prefer, like (e.g., "Pizza")
preferred_cuisine = df_inspec_filtered[df_inspec_filtered['CUISINE DESCRIPTION'] == 'Pizza'].copy()

# Create a new data frame with a DateTimeIndex and a count of inspections per day
df_calendar = preferred_cuisine.groupby(pd.to_datetime(preferred_cuisine['INSPECTION DATE'])).count()['CUISINE DESCRIPTION'].to_frame()

# Create the calendar plot
calplot.calplot(df_calendar['CUISINE DESCRIPTION'], cmap='YlGn', yearlabel_kws={'fontname':'sans-serif'})

plt.show()

In [ ]:
# for every types of restaurants:

# Filter the data for all cuisines
all_cuisines = df_inspec_filtered.copy()

# Create a new data frame with a DateTimeIndex and a count of inspections per day
df_calendar = all_cuisines.groupby(pd.to_datetime(all_cuisines['INSPECTION DATE'])).count()['CUISINE DESCRIPTION'].to_frame()

# Create the calendar plot
calplot.calplot(df_calendar['CUISINE DESCRIPTION'], cmap='YlGn', yearlabel_kws={'fontname':'sans-serif'})

In [ ]:
# for the closed restaurants:

# Create new dataframe with only closed restaurants
closed_restaurants_df = df_inspec_filtered.loc[df_inspec_filtered['ACTION'] == 'Establishment Closed by DOHMH. Violations were cited in the following area(s) and those requiring immediate action were addressed.'].sort_values('Year', ascending=True)

# Create a new data frame with a DateTimeIndex and a count of inspections per day
df_calendar_closed_restaurants = closed_restaurants_df.groupby(pd.to_datetime(closed_restaurants_df['INSPECTION DATE'])).count()['CUISINE DESCRIPTION'].to_frame()

# Create the calendar plot
calplot.calplot(df_calendar['CUISINE DESCRIPTION'], cmap='YlGn', yearlabel_kws={'fontname':'sans-serif'})

plt.show()

The calendar plots above work wonders in showing the big gap between March 2020 and July 2021 due to the pandemic. It is also easy to see how the colored squares darken progressively moving into 2023, illustrating how the occurrences of inspections increase after the quarantine, to levels greater than before the pandemic. 

## 5.3. Barplots

In [ ]:
# Number of each grade by cuisine type
focus_types = df_inspec_filtered.groupby('CUISINE DESCRIPTION')['CAMIS'].count().sort_values(ascending=False)[:20].index
df_focus = df_inspec_filtered[df_inspec_filtered['CUISINE DESCRIPTION'].isin(focus_types)]

df_stacked_barplot = df_focus.groupby(['CUISINE DESCRIPTION','SCORE_GRADE'])['CAMIS'].count().reset_index()

fig = px.bar(df_stacked_barplot, x='CUISINE DESCRIPTION', y='CAMIS', color='SCORE_GRADE', 
             color_discrete_map={
                'A': '#636EFA',
                'B': '#00CC96',
                'C': '#FFA15A'
             },
             title='Grade count by cuisine')
fig.show()

In [ ]:
# Distribution of grades by cuisine type

focus_types = df_inspec_filtered.groupby('CUISINE DESCRIPTION')['CAMIS'].count().sort_values(ascending=False)[:20].index
df_focus = df_inspec_filtered[df_inspec_filtered['CUISINE DESCRIPTION'].isin(focus_types)]


df_stacked_barplot = df_focus.groupby(['CUISINE DESCRIPTION','SCORE_GRADE'])['CAMIS'].count().reset_index()

total_restaurants = df_focus.groupby('CUISINE DESCRIPTION').size().loc[focus_types].to_frame(name='count')
df_stacked_barplot['RATE'] = df_stacked_barplot.apply(lambda x: x['CAMIS'] / total_restaurants.loc[x['CUISINE DESCRIPTION']]['count'], axis=1)



fig = px.bar(df_stacked_barplot, x='CUISINE DESCRIPTION', y='RATE', color='SCORE_GRADE', 
             color_discrete_map={
                'A': '#636EFA',
                'B': '#00CC96',
                'C': '#FFA15A'
             },
             title='Grade distribution by cuisine')
fig.show()

The bar plots above allow the reader to get information about the performance of different cuisines in a single glance. The first bar plot makes it easy to understand not only the number of grades given in each category, but also the relative size of each cuisine type compared to the others. The second plot is also very informative, illustrating the rate at which each cuisine type receives each grade. This way it becomes which types of restaurants have generally greater chances of being generally "clean" or not. The use of colors makes these plots very easy to read and therefore ideal for the visualization of this kind of data.



In [ ]:
# Get the count for each boro and year for closures
boro_year = df_closures.groupby(['BORO', 'YEARMONTH'])['CAMIS'].count().reset_index()

# Pivot the table to have one column per neighborhood
df_pivot = boro_year.pivot(index='YEARMONTH', columns='BORO', values='CAMIS').fillna(0)

# Get the count of total inspections for each boro and yearmonth
total_restaurants = df_inspec_filtered.groupby(['BORO', 'YEARMONTH']).size().to_frame(name='count').reset_index()

# Join the two dataframes on BORO and YEARMONTH
merged_df = pd.merge(boro_year, total_restaurants, on=['BORO', 'YEARMONTH'])

# Create a new dataframe with the rate of closures for each boro and yearmonth
rate_df = merged_df.copy()
rate_df['rate'] = rate_df['CAMIS'] / rate_df['count']

# Pivot the table to have one column per neighborhood
rate_pivot = rate_df.pivot(index='YEARMONTH', columns='BORO', values='rate').fillna(0)

In [ ]:
from bokeh.models import ColumnDataSource, FactorRange, Legend
from bokeh.plotting import figure#, show
from bokeh.palettes import Category20 as Palette
from bokeh.transform import factor_cmap
from bokeh.io import output_file, show
from math import pi

#output_file("output/closure_rate.html")

source = ColumnDataSource(rate_pivot)
boros = rate_pivot.columns.unique()

p = figure(width=800, height=400, x_range=FactorRange(factors=rate_pivot.index.astype(str)),
        title='Rate of Restaurant Closures by Borough and Year/Month',
          x_axis_label='YEARMONTH', y_axis_label='Relative Frequency')

bar = {}
items = []
colors = Palette[len(rate_pivot.columns)]

for idx, i in enumerate(sorted(boros)):
    bar[i] = p.vbar(x='YEARMONTH', top=i, source=source, color = colors[idx],
                    muted_alpha=0.02, muted=True)
    
    items.append((i, [bar[i]]))

legend = Legend(items=items)
p.xaxis.major_label_orientation = pi/2
p.add_layout(legend, 'left')
p.legend.label_text_font_size = '7pt'
    
p.legend.click_policy="mute" # click policy
show(p) # display plot

In [ ]:
import textwrap

focusviolations = df_inspec_filtered.groupby('VIOLATION DESCRIPTION')['CAMIS'].count().sort_values(ascending=False)[:10].index

# Define figure and axes for subplots
nrows=len(focusviolations)//2

fig, ax = plt.subplots(figsize=(10,15), nrows=nrows, ncols=2);
fig.suptitle('Count per year/month by violation', fontsize=14)
fig.supxlabel('year/month')

for idx,val in enumerate(focusviolations):
    
    data = df_inspec_filtered[df_inspec_filtered['VIOLATION DESCRIPTION'] == val].groupby('YEARMONTH')['CAMIS'].count()
    
    r = idx//2
    c = idx%2

    ax[r][c].bar(data.index, data.values, color='purple', width=0.7)
    
    title_lines = textwrap.wrap(val, width=15)
    title = '\n'.join(title_lines)
    ax[r][c].text(-0.5, 0.5, title, transform=ax[r][c].transAxes,
                  horizontalalignment='left', verticalalignment='center',
                  fontsize=7, fontweight='light')
    plt.sca(ax[r][c])
    plt.tick_params(axis='x', bottom=True, labelbottom=True, labelrotation=90, labelsize=7)
    plt.ylabel("Count")
    
fig.tight_layout()
plt.show()

The bar plots above illustrate the counts of each type of violation for each month in the investigated period. Again, they are successful in illustrating the huge drop in inspections during the quarantine and the resurgence the followed after it was over. In general, all the types of violations seem to follow the same pattern, corresponding with the total number of inspections performed, as it showed in the plot below.

In [ ]:
# All occurrences per year/month
data = df_inspec_filtered.groupby('YEARMONTH')['CAMIS'].count()

plt.bar(data.index, data.values, color='blue', width=0.7)
plt.title('Occurrences per year/month')
plt.xticks(rotation=90,fontsize=7)
plt.show()

## 6. Discussion. Think critically about your creation ##

### 6.1 *What went well?* ###

The dataset that was chosen ended up being very useful, as it contains a lot of usable and relevant information. Some data processing and cleaning was needed, but in general the dataset was of good quality and easy to work with. The subject in question is directly relevant to everyday life, and therefore may seem interesting to a lot of people, which is indicative of the value of any piece of information. The visualizations that were used were chosen after careful consideration in order to communicate the desired story as best as possible, and in general they are successful in doing so. The purpose of this project is to make a story about the restaurant scene of New York from the aspect of the restaurants' performance in terms of cleanliness and hygiene, so that the reader can get a clear picture of what is actually happening in NY and can use this information to make safe choices in the future. The end result is an informative and interactive article that we hope is as fun to the reader as it is to us!


### 6.2 *What is still missing? What could be improved?, Why?*

An interesting addition to the analysis would be a combination of the inspection dataset with data about the consumer's reviews on restaurants, and an investigation of how they might correlate. Another idea would be to look into the progression of inspection results in a longer time horizon, e.g. the last 30 years. This way we might find patterns about how inspecting strategies might have evolved through time, and even whether the restaurants themselves are operating better or worse than in the past.




## 7. Contributions. Who did what?

For the greatest part of the assignment we worked together and as a group, keeping everyone involved in every part of the project. Initially we all contributed with suggestions to the discussion about choosing the dataset. After that, Georgios did the greatest part of Data exploration, loading, cleaning and basic barplots taking into account everyone's opinion. Laura took the lead in creating the interactive plots and maps and implemented everything in the website, while together with Charalampos decided which visuals would be part of the story. Charalampos developed the form of the narrative and the structure of the article. Then we all together created the final form of the notebook, making sure everyone's ideas are included and nothing is left out.

|                                              | Charalampos | Georgios | Laura |
|----------------------------------------------|-------------|----------|-------|
| Data loading, cleaning and basic exploration | 15%         | 70%      | 15%   |
| Basic barplots and calendar plots            | 15%         | 70%      | 15%   |
| Interactive plots and maps                   | 15%         | 15%      | 70%   |
| Website implementation                       | 15%         | 15%      | 70%   |
| Background research and story telling        | 70%         | 15%      | 15%   |
| Narrative and structure of the article       | 70%         | 15%      | 15%   |